In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
########### 2a Personen mit >70 Bildern #############
import os, tarfile
from urllib.request import urlretrieve
if not os.path.isfile('lfw-funneled.tgz'):
    print("Downloading")
    urlretrieve('http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz',filename = 'lfw-funneled.tgz')
    
# Pfad zur heruntergeladenen tgz-Datei
tgz_file = 'lfw-funneled.tgz'

# Zielverzeichnis für die Extraktion
extracted_directory = 'lfw_funneled'

# Extrahiere den Datensatz aus der tgz-Datei
if not os.path.exists(extracted_directory):
    with tarfile.open(tgz_file, 'r:gz') as tar:
        print("Extracting...")
        tar.extractall(os.getcwd())
else: 
    print("Verzeichnis gefunden.")

Verzeichnis gefunden.


In [3]:
# Funktion zum Durchsuchen des Verzeichnisses und Ermitteln der Personen mit mindestens 70 Bildern
person_dir_paths = []

def find_persons_with_min_images(dataset_directory, min_images=70):
    persons = {}
    for root, dirs, files in os.walk(dataset_directory):
        if len(files) >= min_images:
            person_name = os.path.basename(root)
            person_dir_paths.append(root)
            persons[person_name] = len(files)

    return {k: v for k, v in persons.items() if v >= min_images}


persons = find_persons_with_min_images(extracted_directory, min_images=70)

# Ausgabe der gefundenen Personen
for person, num_images in persons.items():
    print(f'{person}: {num_images} Bilder')

Ariel_Sharon: 77 Bilder
Colin_Powell: 236 Bilder
Donald_Rumsfeld: 121 Bilder
George_W_Bush: 530 Bilder
Gerhard_Schroeder: 109 Bilder
Hugo_Chavez: 71 Bilder
Tony_Blair: 144 Bilder


In [4]:
from os import listdir
from sklearn.model_selection import train_test_split

image_paths = []
image_name_indexes = []

# Search in everys Person folder for images and append them to the lists
for i, person_dir_path in enumerate(person_dir_paths):
    files = sorted(listdir(person_dir_path))
    for file in files:
        image_name_indexes.append(i)
        image_paths.append(f'{person_dir_path}/{file}')

# Split the data into training and test data
x_train_img_path, x_test_img_path, y_training_name_index, y_test_name_index = train_test_split(image_paths, image_name_indexes, test_size=0.4, random_state=0)

print('Splitted into', len(x_test_img_path), 'test images and', len(x_train_img_path), 'training images.')

Splitted into 516 test images and 772 training images.


In [6]:
# 2b
from skimage import io, transform

def transform_image(img):
    # Bildmitte berechnen
    height, width = img.shape[:2]
    center_x, center_y = width // 2, height // 2
    
    # Ausschnittsgröße festlegen
    crop_size = 96
    
    # Berechne den Ausschnitt
    crop_x1 = center_x - (crop_size // 2)
    crop_x2 = center_x + (crop_size // 2)
    crop_y1 = center_y - (crop_size // 2)
    crop_y2 = center_y + (crop_size // 2)
    
    # Den Ausschnitt ausschneiden
    img = img[crop_y1:crop_y2, crop_x1:crop_x2]
    img = transform.resize(img, (32, 32))
    return img.flatten()

x_train_images = []
x_train_transformed = []

for train_img_path in x_train_img_path:
    img = io.imread(train_img_path, as_gray=True)
    x_train_images.append(img)
    img = transform_image(img)
    x_train_transformed.append(img)

design_matrix = np.array(x_train_transformed)

x_test_images = []
x_test_transformed = []

for test_img_path in x_test_img_path:
    img = io.imread(test_img_path, as_gray=True)
    x_test_images.append(img)
    img = transform_image(img)
    x_test_transformed.append(img)

test_matrix = np.array(x_test_transformed)

df_train = pd.DataFrame(design_matrix)
df_test = pd.DataFrame(test_matrix)

print("Shape Design Matrix: ", design_matrix.shape)
print("Shape Test Matrix: ", test_matrix.shape)
df_train.head()

Shape Design Matrix:  (772, 1024)
Shape Test Matrix:  (516, 1024)


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.500715,0.479520,0.687868,0.730689,0.747468,0.762788,0.730225,0.711743,0.713743,0.687237,...,0.496647,0.485998,0.551722,0.622947,0.556574,0.477024,0.452327,0.431730,0.402049,0.410402
1,0.510545,0.520284,0.636516,0.682492,0.711208,0.724303,0.729710,0.718356,0.743739,0.775675,...,0.361003,0.349891,0.328854,0.260119,0.263067,0.291812,0.313924,0.288129,0.239287,0.355265
2,0.436124,0.488102,0.553063,0.598777,0.652305,0.691400,0.735770,0.784173,0.810917,0.813457,...,0.490018,0.401328,0.385687,0.373462,0.306952,0.303441,0.348612,0.364257,0.447986,0.681894
3,0.350791,0.320254,0.411745,0.520783,0.566392,0.570777,0.593325,0.589648,0.604941,0.613002,...,0.350179,0.376242,0.317607,0.238791,0.171996,0.142533,0.143300,0.133569,0.132797,0.146890
4,0.393305,0.395044,0.394139,0.424829,0.463218,0.511653,0.517516,0.497895,0.494390,0.566904,...,0.474702,0.490612,0.563380,0.549045,0.578195,0.643133,0.611017,0.533875,0.316472,0.318593


In [15]:
###### PCA 2c ######
from sklearn import decomposition

pca = decomposition.PCA(n_components=7, whiten=True)
pca.fit(design_matrix)

train_scores = pca.transform(design_matrix)
test_scores = pca.transform(test_matrix)

print("Shape Train Scores: ", train_scores.shape)
###### PCA Diese Funktion funktioniert nicht, bzw kommen schlechtere ergebnisse ######
# def pca(matrix):
    
#     # Zentrierung
#     matrix = (matrix - matrix.mean()) / matrix.std()
    
#     # SVD
#     U, D, V = np.linalg.svd(matrix)
#     eigenvectors = V
#     print("Eigenvectores shape: ", eigenvectors.shape)
#     eigenfaces = eigenvectors[:7]
#     print("Eigenfaces shape: ", eigenfaces.shape)
    
#     # Transform the data to the first 7 eigenfaces
#     scores = pd.DataFrame(np.matmul(matrix, eigenfaces.T))
#     return scores

# train_scores = pca(design_matrix)
# test_scores = pca(test_matrix)

Eigenvectores shape:  (1024, 1024)
Eigenfaces shape:  (7, 1024)
Eigenvectores shape:  (1024, 1024)
Eigenfaces shape:  (7, 1024)


In [16]:
def label_indexes(indexes):
    # Because Bush is on the index 3 counting from 0 (4th person in List)
    return [1 if index == 3 else -1 for index in indexes]

def evaluate_prediction(prediction_indexes, observation_indexes):
    prediction_labels = label_indexes(prediction_indexes)
    observation_labels = label_indexes(observation_indexes)

    true_positives = 0
    true_negatives = 0
    false_negatives = 0
    false_positives = 0
    dates = 0

    for i in range(len(prediction_indexes)):
        dates += 1
        prediction_label = prediction_labels[i]
        observation_label = observation_labels[i]
        if prediction_label == 1 and observation_label == 1:
            true_positives += 1
        if prediction_label == 1 and observation_label == -1:
            false_positives += 1
        if prediction_label == -1 and observation_label == -1:
            true_negatives += 1
        if prediction_label == -1 and observation_label == 1:
            false_negatives += 1
    


    print('True Positives Rate:', round(100 * true_positives / dates), '%')
    print('False Negatives Rate:', round(100 * false_negatives / dates), '%')
    print('True Negatives Rate:', round(100 * true_negatives / dates), '%')
    print('False Positives Rate:', round(100 * false_positives / dates), '%')

In [17]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

# Test data
print('Test data')
y_test_prediction = gnb.fit(train_scores, y_training_name_index).predict(test_scores)
evaluate_prediction(y_test_prediction, y_test_name_index)
print('\n')

# Training data
print('Training data')
y_train_prediction = gnb.fit(train_scores, y_training_name_index).predict(train_scores)
evaluate_prediction(y_train_prediction, y_training_name_index)

Test data
True Positives Rate: 11 %
False Negatives Rate: 32 %
True Negatives Rate: 47 %
False Positives Rate: 10 %


Training data
True Positives Rate: 34 %
False Negatives Rate: 6 %
True Negatives Rate: 30 %
False Positives Rate: 30 %


In [ ]:
# See: https://github.com/tigju/Naive-Bayes-Classifier-from-scratch/blob/main/naive_bayes.ipynb
class NaiveBayesClassifier():

    def calc_prior(self, features, target):
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()
        return self.prior

    def calc_statistics(self, features, targets):
        self.mean = features.groupby(targets).apply(np.mean).to_numpy()
        self.var = features.groupby(targets).apply(np.var).to_numpy()
        return self.mean, self.var

    def gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob

    def calc_posterior(self, x):
        posteriors = []
        for i in range(self.count):
            prior = np.log(self.prior[i])
            conditional = np.sum(np.log(self.gaussian_density(i, x)))
            posterior = prior + conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]

    def fit(self, features, targets):
        self.classes = np.unique(targets)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        self.calc_statistics(features, targets)
        self.calc_prior(features, targets)

    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

nbc = NaiveBayesClassifier()

# Test data
print('Test data')
x_training_projections_df = pd.DataFrame(train_scores)
y_training_name_index_df = pd.DataFrame(y_training_name_index)[0]
x_test_projections_df = pd.DataFrame(test_scores)

nbc.fit(x_training_projections_df, y_training_name_index_df)
y_test_prediction = nbc.predict(x_test_projections_df)
evaluate_prediction(y_test_prediction, y_test_name_index)
print('\n')

# Training data
print('Training data')
x_training_projections_df = pd.DataFrame(train_scores)
y_training_name_index_df = pd.DataFrame(y_training_name_index)[0]
x_training_projections_df = pd.DataFrame(train_scores)

nbc.fit(x_training_projections_df, y_training_name_index_df)
y_training_prediction = nbc.predict(x_training_projections_df)
evaluate_prediction(y_training_prediction, y_training_name_index)

Test data
True Positives Rate: 39 %
False Negatives Rate: 4 %
True Negatives Rate: 20 %
False Positives Rate: 37 %


Training data
True Positives Rate: 36 %
False Negatives Rate: 4 %
True Negatives Rate: 21 %
False Positives Rate: 39 %


c:\Users\marco\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\marco\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
